In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
pd.set_option('display.max_rows', None)

In [6]:
boe=pd.read_csv('../data/BOE_def.csv')

In [45]:
testeo=pd.read_csv('../data/BOE_Sin_Puja.csv')

testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   identificador         6154 non-null   object
 1   tipo_de_subasta       6154 non-null   object
 2   cuenta_expediente     6154 non-null   object
 3   fecha_de_inicio       6154 non-null   object
 4   fecha_de_conclusion   6154 non-null   object
 5   cantidad_reclamada    6154 non-null   int64 
 6   anuncio_BOE           6154 non-null   object
 7   valor_subasta         6154 non-null   int64 
 8   tasacion              6154 non-null   int64 
 9   puja_minima           6154 non-null   int64 
 10  tramos_entre_pujas    6154 non-null   int64 
 11  importe_del_deposito  6154 non-null   int64 
 12  direccion             6154 non-null   object
 13  codigo_postal         6154 non-null   int64 
 14  localidad             6154 non-null   object
 15  provincia             6154 non-null   

In [46]:
boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Columns: 2180 entries, cantidad_reclamada to codigo_postal_52001
dtypes: int64(5), uint8(2175)
memory usage: 13.0 MB


In [7]:
import regex as re

In [8]:
def limpiar_(fecha_de_inicio):
    fecha_de_inicio=re.findall('\d{4}', fecha_de_inicio)
    fecha_de_inicio=' '.join(fecha_de_inicio)
    return fecha_de_inicio

In [9]:
boe.fecha_de_inicio=boe.fecha_de_inicio.apply(limpiar_)

In [10]:
boe.fecha_de_conclusion=boe.fecha_de_conclusion.apply(limpiar_)

In [11]:
boe.head()

,identificador,tipo_de_subasta,cuenta_expediente,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,anuncio_BOE,valor_subasta,tasacion,puja_minima,tramos_entre_pujas,importe_del_deposito,direccion,codigo_postal,localidad,provincia,puja,pm2
0,SUB-JA-2022-204013,JUDICIAL EN VÍA DE APREMIO,4743 0000 05 1018 20,2022,2022,7145,BOE-B-2022-35270,108482,0,0,2169,5424,Urbanización Els Racons nº 5. partida Michafiga,46220,Picassent,Valencia/València,58580,1088
1,SUB-JA-2022-196067,JUDICIAL EN VÍA DE APREMIO,2368 0000 06 0191 20,2022,2022,142298,BOE-B-2022-35269,305832,0,0,6116,15291,"CALLE GABRIEL GARCÍA MÁRQUEZ, 31",28980,PARLA,Madrid,177382,1721
2,SUB-JA-2022-204060,JUDICIAL EN VÍA DE APREMIO,2995 0000 06 0456 16,2022,2022,55933,BOE-B-2022-35263,247902,0,0,0,12395,VIVIENDA 1 BLOQUE 19 CONJUNTO RESIDENCIAL HACI...,29600,Marbella,Málaga,252000,1721
3,SUB-JA-2022-198972,JUDICIAL EN VÍA DE APREMIO,3997 0000 06 1661 20,2022,2022,210425,BOE-B-2022-28879,826272,0,0,16525,41313,"AVENIDA DE JACARANDA, 5, URBANIZACION CLUB DE ...",41930,BORMUJOS,Sevilla,512288,1557
4,SUB-JA-2022-203383,JUDICIAL EN VÍA DE APREMIO,5418 0000 06 0169 21,2022,2022,34853,BOE-B-2022-35253,121684,121684,45978,2433,6084,"Calle Palma de Mallorca, nº 13, bloque 19, 3º ...",35016,Las Palmas de Gran Canaria,Las Palmas,45978,1130


In [12]:
boe.drop(columns=['identificador','tipo_de_subasta','cuenta_expediente',
                  'anuncio_BOE','tasacion','puja_minima','tramos_entre_pujas',
                 'importe_del_deposito','direccion','localidad'], inplace=True)

In [13]:
boe.head()

,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2
0,2022,2022,7145,108482,46220,Valencia/València,58580,1088
1,2022,2022,142298,305832,28980,Madrid,177382,1721
2,2022,2022,55933,247902,29600,Málaga,252000,1721
3,2022,2022,210425,826272,41930,Sevilla,512288,1557
4,2022,2022,34853,121684,35016,Las Palmas,45978,1130


In [14]:
boe.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fecha_de_inicio      6154 non-null   object
 1   fecha_de_conclusion  6154 non-null   object
 2   cantidad_reclamada   6154 non-null   int64 
 3   valor_subasta        6154 non-null   int64 
 4   codigo_postal        6154 non-null   int64 
 5   provincia            6154 non-null   object
 6   puja                 6154 non-null   int64 
 7   pm2                  6154 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 1.4 MB


In [15]:
boe['fecha_de_inicio'] = boe['fecha_de_inicio'].astype(int)

In [16]:
boe['fecha_de_conclusion'] = boe['fecha_de_conclusion'].astype(int)

In [17]:
#eliminamos esta columna porque el precio se cierra en la fecha de conclusion y varia segun el año final.
boe.drop(columns=['fecha_de_inicio'], inplace=True)

In [ ]:
#eliminamos codigo postal? con eso hemos sacado el pm2, no nos interesa más. Ademas contamos con la provincia.
#boe.drop(columns=['codigo_postal'], inplace=True)

In [18]:
#creamos new colum con la diferencia entre la deuda(cantidad_reclamada) y el valor que le da el juez(valor_subasta)
boe['difference'] = boe['valor_subasta'].sub(boe['cantidad_reclamada'], axis = 0)

In [19]:
boe.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2,difference
0,2022,7145,108482,46220,Valencia/València,58580,1088,101337
1,2022,142298,305832,28980,Madrid,177382,1721,163534
2,2022,55933,247902,29600,Málaga,252000,1721,191969
3,2022,210425,826272,41930,Sevilla,512288,1557,615847
4,2022,34853,121684,35016,Las Palmas,45978,1130,86831


In [ ]:
#boe.drop(columns=['cantidad_reclamada','valor_subasta'], inplace=True)

In [20]:
boe=pd.get_dummies(boe, columns=['fecha_de_conclusion','provincia','codigo_postal'], drop_first=True)

boe.head()

cantidad_reclamada  valor_subasta    puja   pm2  difference  \
0                7145         108482   58580  1088      101337   
1              142298         305832  177382  1721      163534   
2               55933         247902  252000  1721      191969   
3              210425         826272  512288  1557      615847   
4               34853         121684   45978  1130       86831   

   fecha_de_conclusion_2017  fecha_de_conclusion_2018  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2019  fecha_de_conclusion_2020  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2021  fecha_de_conclusion_2022  provincia_Albacete  \
0                         0                         1                   0   
1                         0                         1                   0   
2                         0                         1                   0   
3                         0                         1                   0   
4                         0                         1                   0   

   provincia_Alicante  provincia_Alicante/Alacant  provincia_Almería  \
0                   0                           0                  0   
1                   0                           0                  0   
2                   0                           0                  0   
3                   0                           0                  0   
4                   0                           0                  0   

   provincia_Araba/Álava  provincia_Asturias  provincia_Badajoz  \
0                      0                   0                  0   
1                      0                   0                  0   
2                      0                   0                  0   
3                      0                   0                  0   
4                      0                   0                  0   

   provincia_Baleares  provincia_Barcelona  provincia_Bizkaia  \
0                   0                    0                  0   
1                   0                    0                  0   
2                   0                    0                  0   
3                   0                    0                  0   
4                   0                    0                  0   

   provincia_Burgos  provincia_Cantabria  provincia_Castellón  \
0                 0                    0                    0   
1                 0                    0                    0   
2                 0                    0                    0   
3                 0                    0                    0   
4                 0                    0                    0   

   provincia_Castellón/Castelló  provincia_Ceuta  provincia_Ciudad Real  \
0                             0                0                      0   
1                             0                0                      0   
2                             0                0                      0   
3                             0                0                      0   
4                             0                0                      0   

   provincia_Cuenca  provincia_Cáceres  provincia_Cádiz  provincia_Córdoba  \
0                 0                  0                0                  0   
1                 0                  0                0                  0   
2                 0                  0                0                  0   
3                 0                  0                0                  0  

In [21]:
boe.pm2=LabelEncoder().fit_transform(boe.pm2)

In [22]:
boe.cantidad_reclamada=LabelEncoder().fit_transform(boe.cantidad_reclamada)

In [23]:
boe.valor_subasta=LabelEncoder().fit_transform(boe.valor_subasta)

In [24]:
boe.difference=LabelEncoder().fit_transform(boe.difference)

In [ ]:
#boe=boe.rename(columns = {'diff':'diference'})

In [25]:
boe_x=boe.drop('puja', axis=1)
boe_y=boe.puja

In [29]:
X_train, X_test, y_train, y_test = tts(boe_x, boe_y,train_size=0.9, test_size=0.1,random_state =71)

**Iniciamos los modelos**

In [30]:
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()

**Entrenamos modelos**

In [31]:
linreg.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
svr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)

Learning rate set to 0.053657
0:	learn: 584915.1604973	total: 64.4ms	remaining: 1m 4s
1:	learn: 578347.3759690	total: 67.1ms	remaining: 33.5s
2:	learn: 571884.1289702	total: 69.8ms	remaining: 23.2s
3:	learn: 565747.4920656	total: 70.9ms	remaining: 17.6s
4:	learn: 559617.8710251	total: 73.2ms	remaining: 14.6s
5:	learn: 553670.8190351	total: 75.6ms	remaining: 12.5s
6:	learn: 547819.6953067	total: 78.7ms	remaining: 11.2s
7:	learn: 542011.8769577	total: 81.5ms	remaining: 10.1s
8:	learn: 536352.3865506	total: 84.3ms	remaining: 9.28s
9:	learn: 530725.0263818	total: 87.2ms	remaining: 8.63s
10:	learn: 525070.8698431	total: 90ms	remaining: 8.09s
11:	learn: 519600.5681040	total: 92.9ms	remaining: 7.64s
12:	learn: 514262.8863618	total: 95.6ms	remaining: 7.25s
13:	learn: 509019.0730918	total: 98.7ms	remaining: 6.95s
14:	learn: 504956.9174998	total: 102ms	remaining: 6.68s
15:	learn: 499920.0928444	total: 105ms	remaining: 6.44s
16:	learn: 494950.9758853	total: 108ms	remaining: 6.24s
17:	learn: 48993

178:	learn: 129120.5780624	total: 588ms	remaining: 2.7s
179:	learn: 127970.4493508	total: 591ms	remaining: 2.69s
180:	learn: 127625.1615448	total: 594ms	remaining: 2.69s
181:	learn: 126510.1676830	total: 597ms	remaining: 2.68s
182:	learn: 125446.7818271	total: 600ms	remaining: 2.68s
183:	learn: 124696.7657981	total: 603ms	remaining: 2.67s
184:	learn: 124361.7949822	total: 606ms	remaining: 2.67s
185:	learn: 123405.9252369	total: 609ms	remaining: 2.66s
186:	learn: 122571.9594571	total: 612ms	remaining: 2.66s
187:	learn: 121675.3108296	total: 615ms	remaining: 2.65s
188:	learn: 120978.0258684	total: 618ms	remaining: 2.65s
189:	learn: 120293.1488708	total: 621ms	remaining: 2.65s
190:	learn: 119626.6011798	total: 623ms	remaining: 2.64s
191:	learn: 118877.5499319	total: 626ms	remaining: 2.64s
192:	learn: 118212.6829012	total: 629ms	remaining: 2.63s
193:	learn: 117590.3074198	total: 632ms	remaining: 2.63s
194:	learn: 116792.1815309	total: 635ms	remaining: 2.62s
195:	learn: 116185.5971414	total

384:	learn: 72137.3200651	total: 1.19s	remaining: 1.89s
385:	learn: 72050.3795324	total: 1.2s	remaining: 1.91s
386:	learn: 72022.3263532	total: 1.21s	remaining: 1.91s
387:	learn: 71995.9168152	total: 1.21s	remaining: 1.91s
388:	learn: 71951.5409308	total: 1.21s	remaining: 1.9s
389:	learn: 71801.3599530	total: 1.21s	remaining: 1.9s
390:	learn: 71747.8766497	total: 1.22s	remaining: 1.9s
391:	learn: 71590.9335783	total: 1.22s	remaining: 1.89s
392:	learn: 71460.0013719	total: 1.22s	remaining: 1.89s
393:	learn: 71432.7360002	total: 1.23s	remaining: 1.89s
394:	learn: 71375.3357550	total: 1.23s	remaining: 1.88s
395:	learn: 71348.6961516	total: 1.23s	remaining: 1.88s
396:	learn: 71292.9692375	total: 1.24s	remaining: 1.88s
397:	learn: 71236.9833940	total: 1.24s	remaining: 1.87s
398:	learn: 71210.9002928	total: 1.24s	remaining: 1.87s
399:	learn: 71157.2293909	total: 1.24s	remaining: 1.86s
400:	learn: 71132.9009655	total: 1.25s	remaining: 1.86s
401:	learn: 71012.6823765	total: 1.25s	remaining: 1.

586:	learn: 62054.3706312	total: 1.78s	remaining: 1.25s
587:	learn: 62042.7838075	total: 1.78s	remaining: 1.25s
588:	learn: 61986.3927990	total: 1.78s	remaining: 1.25s
589:	learn: 61974.9454708	total: 1.79s	remaining: 1.24s
590:	learn: 61929.1536248	total: 1.79s	remaining: 1.24s
591:	learn: 61917.1964124	total: 1.79s	remaining: 1.24s
592:	learn: 61905.8663438	total: 1.8s	remaining: 1.23s
593:	learn: 61834.4300250	total: 1.8s	remaining: 1.23s
594:	learn: 61789.6334693	total: 1.8s	remaining: 1.23s
595:	learn: 61770.4805559	total: 1.8s	remaining: 1.22s
596:	learn: 61751.9038591	total: 1.81s	remaining: 1.22s
597:	learn: 61733.8814980	total: 1.81s	remaining: 1.22s
598:	learn: 61722.0819510	total: 1.81s	remaining: 1.21s
599:	learn: 61618.2714136	total: 1.82s	remaining: 1.21s
600:	learn: 61597.2887444	total: 1.82s	remaining: 1.21s
601:	learn: 61586.0585589	total: 1.82s	remaining: 1.2s
602:	learn: 61568.2752023	total: 1.82s	remaining: 1.2s
603:	learn: 61557.1350996	total: 1.83s	remaining: 1.2s

791:	learn: 56524.2690158	total: 2.37s	remaining: 623ms
792:	learn: 56511.6765142	total: 2.38s	remaining: 620ms
793:	learn: 56504.2097917	total: 2.38s	remaining: 617ms
794:	learn: 56496.8202476	total: 2.38s	remaining: 614ms
795:	learn: 56489.4741806	total: 2.38s	remaining: 611ms
796:	learn: 56482.4108448	total: 2.39s	remaining: 608ms
797:	learn: 56470.3444778	total: 2.39s	remaining: 605ms
798:	learn: 56452.4808437	total: 2.39s	remaining: 602ms
799:	learn: 56408.6525750	total: 2.4s	remaining: 599ms
800:	learn: 56370.0739223	total: 2.4s	remaining: 596ms
801:	learn: 56361.9214536	total: 2.4s	remaining: 593ms
802:	learn: 56326.5366018	total: 2.4s	remaining: 590ms
803:	learn: 56292.1849407	total: 2.41s	remaining: 587ms
804:	learn: 56284.2393185	total: 2.41s	remaining: 584ms
805:	learn: 56276.9881767	total: 2.41s	remaining: 581ms
806:	learn: 56246.7557535	total: 2.42s	remaining: 578ms
807:	learn: 56239.5302078	total: 2.42s	remaining: 575ms
808:	learn: 56227.5958824	total: 2.42s	remaining: 57

998:	learn: 53116.7633927	total: 2.97s	remaining: 2.97ms
999:	learn: 53111.6107511	total: 2.97s	remaining: 0us


LGBMRegressor()

In [32]:
from sklearn.metrics import mean_squared_error as mse

In [33]:
from sklearn.metrics import r2_score as r2

In [34]:
# regresion lineal

y_pred=linreg.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(711455.5924279436, -1.0760358932401424)

In [35]:
linreg.score(X_train, y_train), linreg.score(X_test, y_test)


(0.35887764254395693, -1.0760358932401424)

In [36]:
# ridge

y_pred=ridge.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(619396.5996483864, -0.5735367725898042)

In [43]:
## rfr

y_pred=rfr.predict(X_test)

mse(y_test, y_pred, squared=False)  , r2(y_test, y_pred)# RMSE

(257641.5175833379, 0.7277480744294844)

In [44]:
##gbr
y_pred=gbr.predict(X_test)

mse(y_test, y_pred, squared=False) , r2(y_test, y_pred) 

(307874.1637517207, 0.611236258373665)

In [40]:
##xgbr

y_pred=xgbr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(1131163.8397465458, -4.247957264141792)

In [41]:
##ctr
y_pred=ctr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(376866.0476294013, 0.41747671634422845)

In [42]:
## lbbmr
y_pred=lgbmr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(175773.85504769185, 0.8732792255858954)